# Import

In [13]:
import pandas as pd
import json

from operator import itemgetter

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.vectorstores import FAISS
from langchain.schema.retriever import BaseRetriever, Document

from langchain import HuggingFaceTextGenInference

# FAISS

In [6]:
# load docs

df = pd.read_csv('./data/qa.csv')
df.shape

(80, 3)

In [7]:
df.head()

,category,question,answer
0,常見問答集,Q01.子女從姓及改姓有那些規定？又應如何辦理登記？,一、配合民法96年5月23日修正公布，戶政事務所辦理新生兒辦理出生登記時從姓之處理原則：父母...
1,常見問答集,Q02.改名有那些規定？又應如何辦理登記？,一、姓名符合下列情形之一者，當事人可向全國任一戶政事務所申請改名：同時在一公民營事業機構、機...
2,常見問答集,Q03.如何辦理認領登記？,認領登記，以認領人為申請人，認領人不為申請時，以被認領人為申請人，並準備下列文件至任一戶政事...
3,常見問答集,Q04.承租人已搬走但戶籍未遷出，可否將其戶籍遷出？,一、按行政法院56年判字第60號判例：「遷徙係事實行為，遷徙登記自應依事實認定之。」次按戶籍...
4,常見問答集,Q05.如何辦理初設戶籍登記？,一、申請人:（一）當事人為未成年人：由法定代理人（父母雙方）、戶長。（二）當事人為成年人：本...


In [10]:
# 將他轉為Document

docs = []

for idx, row in df.iterrows():
    doc_json = {
        "Q": row.question,
        "A": row.answer
    }
    doc_content = json.dumps(doc_json, ensure_ascii=False)
    
    # 索引儲存的 Document 內容
    current_document = Document(
        page_content=doc_content,  # 主要內文
        metadata={
            'source': '中華民國內政部戶政司全球資訊網',
        }
    )
    docs.append(current_document)

In [11]:
docs[:3]

[Document(page_content='{"Q": "Q01.子女從姓及改姓有那些規定？又應如何辦理登記？", "A": "一、配合民法96年5月23日修正公布，戶政事務所辦理新生兒辦理出生登記時從姓之處理原則：父母已約定子女從父姓或母姓者，提憑父母約定書辦理。父母約定不成者，由申請人於戶政事務所抽籤決定該子女依父姓或母姓登 記。逾期未辦理出生登記，經催告仍未申請者，由戶政事務所抽籤決定該子女依 父姓或母姓登記。非婚生子女，依母姓登記。無依兒童且父母身分不明者，依監護人之姓氏登記。經戶政事務所抽籤決定姓氏者或無依兒童依監護人之姓氏登記者，嗣後變更 姓氏，不計入民法第1059條第4項次數之計算。新生兒出生前或出生登記前，父母一方死亡，由生存之一方單方決定子女姓 氏，又父母均死亡，由監護人決定該子女從父姓或從母姓，心神喪失或精神耗弱者，比照辦理。申請人抽籤後不辦理出生登記，於逕為出生登記前，得再抽籤決定。另父母 一方行方不明、雙方不往來或拒絕約定子女姓氏，得依約定不成之情形，由申請人抽籤決定新生兒依父姓或母姓登記。父母已約定子女從父姓或母姓者，提憑父母約定書辦理。父母約定不成者，由申請人於戶政事務所抽籤決定該子女依父姓或母姓登 記。逾期未辦理出生登記，經催告仍未申請者，由戶政事務所抽籤決定該子女依 父姓或母姓登記。非婚生子女，依母姓登記。無依兒童且父母身分不明者，依監護人之姓氏登記。經戶政事務所抽籤決定姓氏者或無依兒童依監護人之姓氏登記者，嗣後變更 姓氏，不計入民法第1059條第4項次數之計算。新生兒出生前或出生登記前，父母一方死亡，由生存之一方單方決定子女姓 氏，又父母均死亡，由監護人決定該子女從父姓或從母姓，心神喪失或精神耗弱者，比照辦理。申請人抽籤後不辦理出生登記，於逕為出生登記前，得再抽籤決定。另父母 一方行方不明、雙方不往來或拒絕約定子女姓氏，得依約定不成之情形，由申請人抽籤決定新生兒依父姓或母姓登記。二、子女經出生登記後，申請改姓相關規定：未成年前，得由父母以書面約定，變更為父姓或母姓；成年後，得自行申請 變更為父姓或母姓。父母離婚、一方死亡、生死不明滿3年或顯有未盡保護或教養義務之情事 者，為子女之利益，得請求法院宣告變更子女之姓氏為父姓或母姓。父母變更姓氏後，已從其姓之直系血親卑親屬應隨同改姓。未成年前，得由父母以書面約定，變更為父姓或母姓；成年

In [17]:
# Embeddins
modelPath = "THUDM/chatglm3-6b"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cuda:0'}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
)

vectorstore = FAISS.from_documents(
    docs, embedding=embeddings
)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

No sentence-transformers model found with name /home/B20711/.cache/torch/sentence_transformers/THUDM_chatglm3-6b. Creating a new one with MEAN pooling.
Loading checkpoint shards: 100%|██████████| 7/7 [00:07<00:00,  1.09s/it]


ValueError: Tokenizer class ChatGLMTokenizer does not exist or is not currently imported.

# 設定 Document Combine 的格式

In [ ]:
from langchain.prompts.prompt import PromptTemplate
from langchain.schema import format_document

DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")
def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

# 設定丟給LLM的prompt

In [ ]:
template = """<|system|>
根據以下 Contex 資料回答 User Question。

規則：
1.從 Context 比對所有JSON的欄位「Q」找出與「User Question」在關鍵詞和上下文語意相似的 JSON。
2.如果找不到任何相似的JSON，回覆「查無相關資訊」。
3.必須直接引用相似JSON的「Q」欄位中完整所有文字作為你的答案(必須保留原文中的換行符號（\r\n）與任何格式)不用回傳 「A」，用繁體中文並且不做任何修改。

Contex:
{context}

<|user|>
{question} 
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = (
    {"context": retriever | _combine_documents, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
chain = (
    {"context": retriever | _combine_documents, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
chain.invoke("Q01.子女從姓及改姓有那些規定？又應如何辦理登記？")